In [1]:
!pip uninstall scikit-learn -y

Uninstalling scikit-learn-0.22.2.post1:
  Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
!pip install -U scikit-learn

     |████████████████████████████████| 22.3MB 67.3MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pylab as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
import gc
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import *
from dateutil import parser
from datetime import datetime
from sklearn.linear_model import LinearRegression
from itertools import product
import torch
from torch import nn, from_numpy
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from scipy.sparse import coo_matrix, hstack

In [5]:
def rmse(y_true, y_pred):

    return mean_squared_error(
            y_true=y_true,
            y_pred=y_pred,
            squared=False
        )

In [6]:
df_train = pd.read_csv(r'/content/drive/MyDrive/regression/train.csv')
df_test = pd.read_csv(r'/content/drive/MyDrive/regression/test.csv')
df_submission = pd.read_csv(r'/content/drive/MyDrive/regression/submission.csv')
df_store = pd.read_csv(r'/content/drive/MyDrive/regression/store.csv')
df_article = pd.read_csv(r'/content/drive/MyDrive/regression/article.csv')
df_sales_count = pd.read_csv(r'/content/drive/MyDrive/regression/sales_count.csv')
df_holidays = pd.read_csv(r'/content/drive/MyDrive/regression/holidays.csv')

Таблицы article и store содержат категориальные признаки, информация о которых содержится в столбцах ARTICLE_ID и STORE_ID. Поэтому эти признаки излишние, что подтверждают эксперименты с линейной регрессией.

Добавим в качестве признака день недели и построим простой бэйзлайн - линейную регрессию на все имеющиеся признаки.

In [ ]:
df_train = pd.merge(df_train,df_sales_count)
df_train['IS_HOLIDAY'] = df_train['DAY_ID'].apply(lambda x: x in df_holidays['DAY_ID'].values)
df_train['WEEKDAY'] = df_train.DAY_ID.apply(lambda x: datetime.strptime(x,'%Y-%m-%d').weekday())

Разобьем данные для обучения и валидации. В качестве валидационного датасета возьмем 15 последних дней из обучающей выборки.

In [ ]:
X_train,y_train,X_val,y_val = df_train[df_train['DAY_ID']<='2017-07-16'].drop(['DAY_ID','SALES'],axis=1),df_train[df_train['DAY_ID']<='2017-07-16']['SALES'].values,\
df_train[df_train['DAY_ID']>'2017-07-16'].drop(['DAY_ID','SALES'],axis=1),df_train[df_train['DAY_ID']>'2017-07-16']['SALES'].values

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')

In [ ]:
X_train_trans = hstack([enc.fit_transform(X_train.drop(['SALES_COUNT'],axis=1)),coo_matrix(X_train[['SALES_COUNT']].values)])
X_val_trans = hstack([enc.transform(X_val.drop(['SALES_COUNT'],axis=1)),coo_matrix(X_val[['SALES_COUNT']].values)])

In [ ]:
reg = LinearRegression().fit(X_train_trans,y_train)

In [ ]:
print("Результаты для линейной регрессии:")
print( 'RMSE on train: {}, RMSE on val: {}'.format( rmse(y_train,reg.predict(X_train_trans)), rmse(y_val,reg.predict(X_val_trans)) ) )
print( 'R2 on train: {}, R2 on val: {}'.format( reg.score(X_train_trans,y_train), reg.score(X_val_trans,y_val) ) )

Результаты для линейной регрессии:
RMSE on train: 22.577699607412015, RMSE on val: 21.870995473243475
R2 on train: 0.2078610578010318, R2 on val: 0.2032628995910768


В данной модели не учитывается временная структура данных. Поэтому мы можем, например, использовать реккурентную нейронную сеть LSTM для учета временной структуры данных. Как показали эксперименты, лучше прогнозировать продажи отдельно для каждого магазина, т е создать и обучить нейронную сеть для каждого магазина отдельно. Для каждого магазина прогнозируемой переменной будет вектор из всевозможных товаров (ARTICLE_ID), которые имеются в обучающей или тестовой выборке, причем пропущенные значения заполняются нулями. Оставшиеся признаки (число посетителей магазина, наличие акции на товар и т д) учитываются линейно. Т е модель может быть записанав виде:
$$
y = f(x_1)+w^Tx_2 + \epsilon,
$$
где $x_1$ это лаговые значения целевой переменной, $f$ - нейронная сеть, $x_2$ - вектор оставшихся признаков.

Заполним отсутствующие значение нулями.

In [7]:
articles = {}
for store in range(1,55):
  articles[store]=list(set(df_submission[df_submission.STORE_ID==store].ARTICLE_ID.unique()).union(set(df_train[df_train.STORE_ID==store].ARTICLE_ID.unique())))

In [8]:
date = (np.array('2017-06-01', dtype=np.datetime64) + np.arange(61)).astype(str)
res = None
for store in df_submission.STORE_ID.unique():
  if res is None:
    res = list( product(date,[store],articles[store] ))
  else:
    res.extend( list( product(date,[store],articles[store] )) )

In [9]:
res = pd.DataFrame(res,columns = ['DAY_ID','STORE_ID','ARTICLE_ID'])
res.sort_values(by = ['DAY_ID','STORE_ID','ARTICLE_ID'],inplace=True)
res = pd.merge(res,df_train.drop(['SALES_COUNT','IS_HOLIDAY','WEEKDAY'],axis=1),how='outer',on = ['DAY_ID','STORE_ID','ARTICLE_ID'])
res = pd.merge(res,df_sales_count[df_sales_count.DAY_ID<='2017-07-31'],how='outer')

In [10]:
res['IS_HOLIDAY'] = res['DAY_ID'].apply(lambda x: x in df_holidays['DAY_ID'].values)
res['WEEKDAY'] = res['DAY_ID'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d').weekday())

In [11]:
res.fillna(0.,inplace=True)

Создадим класс, который воспроизводит описанную выше модель.

In [12]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, pred_len,num_layers,dropout=0.):
      super().__init__()

      self.input_size=input_size
      self.hidden_size = hidden_size
      self.pred_len=pred_len
      self.output_size=output_size

      self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,num_layers=num_layers,dropout=dropout,batch_first=True)
      self.linear = nn.Linear(hidden_size*num_layers*2, output_size*pred_len)

      self.actions = nn.Parameter(torch.zeros(output_size).to(device),requires_grad = True)
      self.actions.data.uniform_(-1/output_size, 1/output_size)

      self.holidays = nn.Parameter(torch.zeros(output_size).to(device),requires_grad = True)
      self.holidays.data.uniform_(-1/output_size, 1/output_size)

      self.weekdays = nn.Parameter(torch.zeros(7*output_size).to(device),requires_grad = True)
      self.weekdays.data.uniform_(-1/7/output_size, 1/7/output_size)

      self.sales = nn.Parameter(torch.zeros(output_size).to(device),requires_grad = True)
      self.sales.data.uniform_(-1/output_size, 1/output_size)



    def forward(self, input_seq,actions,weekdays,sales,holidays):
      _,(h_n,c_n) = self.lstm(input_seq)
      res = torch.cat([h_n,c_n])
      res = torch.transpose(res,0,1)
      res = res.reshape(res.shape[0],-1)
      return (self.linear(res)).reshape(-1,self.pred_len,self.output_size)+self.actions[actions]*(actions!=0)+self.holidays[holidays]*(holidays!=0)+self.weekdays[weekdays]+sales*self.sales

Создадим функция, которая подготавливает данные для модели в нужном формате

In [13]:
def inout_seq(TS,A,We,S,H,train_len,pred_len):
  N = TS.shape[0]
  X,y,act,week,s,h = [],[],[],[],[],[]
  for i in range(train_len,N-pred_len):
    
    X.append(np.concatenate([ TS[i-train_len:i],S[i-train_len:i,[0]] ],axis=1) )
    y.append(TS[i:i+pred_len])
    act.append(A[i:i+pred_len])
    week.append(We[i:i+pred_len])
    s.append(S[i:i+pred_len])
    h.append(H[i:i+pred_len])

  return np.stack(X),np.stack(y),np.stack(act),np.stack(week),np.stack(s),np.stack(h)

In [14]:
pd.options.mode.chained_assignment = None
device = torch.device('cuda')

Зафиксируем параметры, которые были подобраны экспериментально

In [15]:
train_len = 7
pred_len = 15
bs = 3
n_epoches = 50
lr = 1e-3
num_layers = 2
dropout = 0.2
hidden_size = 100

Тренировочный цикл

In [ ]:
rmse_dict = {}
se = None
for store in res.STORE_ID.unique():
  ds = res[res.STORE_ID==store]
  ds.sort_values(by=['DAY_ID','ARTICLE_ID'],inplace=True)

  ids = ds.ARTICLE_ID.unique()
  N_items = ids.shape[0]
  id_int = {ids[i]:i for i in range(N_items)}
  int_id = {i:ids[i] for i in range(N_items)}

  ds['ind'] = ds['ARTICLE_ID'].apply(lambda x: id_int[x])
  ds['WEEKDAY'] = ds['WEEKDAY']*N_items+ds['ind']
  ds['IS_ACTION'] = ds['IS_ACTION']*ds['ind']
  ds['IS_HOLIDAY'] = ds['IS_HOLIDAY']*ds['ind']


  TS = ds.SALES.values.reshape(-1,N_items)

  A = ds.IS_ACTION.values.reshape(-1,N_items)
  We = ds.WEEKDAY.values.reshape(-1,N_items)
  S = ds.SALES_COUNT.values.reshape(-1,N_items)
  H = ds.IS_HOLIDAY.values.reshape(-1,N_items)
  
  X,y,A,We,S,H = inout_seq(TS,A,We,S,H,train_len,pred_len)

  X,y,A,We,S,H = from_numpy(X).to(device).float(),from_numpy(y).to(device).float(),from_numpy(A).to(device).long(),from_numpy(We).to(device).long(), \
  from_numpy(S).to(device).float(),from_numpy(H).to(device).long()

  X_train,y_train,A_train,We_train,S_train,H_train = X[:-1],y[:-1],A[:-1],We[:-1],S[:-1],H[:-1]

  X_val,y_val,A_val,We_val,S_val,H_val = X[[-1]],y[[-1]],A[[-1]],We[[-1]],S[[-1]],H[[-1]]

  model = LSTM(input_size=N_items+1, hidden_size=hidden_size, output_size=N_items, pred_len=pred_len,num_layers=num_layers,dropout=dropout).to(device)
  opt = torch.optim.Adam(model.parameters(),lr=lr)
  N = X_train.shape[0]
  crit = nn.MSELoss()
  print("Store #{}".format(store))
  model.eval()
  preds = model(X_val,A_val,We_val,S_val,H_val)
  print('Epoch: {}, RMSE test: {}, R2 test: {}'.format(0, (((preds[y_val!=0]-y_val[y_val!=0])**2).mean()**0.5).item() ,
                                              r2_score(y_val[y_val!=0].cpu().detach().numpy(),preds[y_val!=0].cpu().detach().numpy() ) ) )
  print('\n')

  for epoch in range(n_epoches):
    model.train()
    inds = torch.randperm(N).to(device)
    for i in range(N//bs):
      opt.zero_grad()
      X_batch,y_batch,A_batch,We_batch,S_batch,H_batch = X_train[inds[i*bs:(i+1)*bs]],y_train[inds[i*bs:(i+1)*bs]],A_train[inds[i*bs:(i+1)*bs]],We_train[inds[i*bs:(i+1)*bs]], \
      S_train[inds[i*bs:(i+1)*bs]], H_train[inds[i*bs:(i+1)*bs]]

      preds = model(X_batch,A_batch,We_batch,S_batch,H_batch)
      loss = crit(preds*(y_batch!=0),y_batch)
      loss.backward()
      opt.step()
    if epoch%(n_epoches//2)==0 or epoch==n_epoches-1:
      model.eval()
      preds = model(X_train,A_train,We_train,S_train,H_train)
      rmse = ( ((preds-y_train)**2)[y_train!=0].mean()**0.5).item()

      print('Epoch: {}, RMSE train: {}, R2 train: {}'.format(epoch+1, rmse ,
                                                r2_score(y_train[y_train!=0].cpu().detach().numpy(),preds[y_train!=0].cpu().detach().numpy() ) ) )
      
      preds = model(X_val,A_val,We_val,S_val,H_val)
      rmse = ( ((preds-y_val)**2)[y_val!=0].mean()**0.5).item()
      print('Epoch: {}, RMSE test: {}, R2 test: {}'.format(epoch+1, rmse ,
                                                r2_score(y_val[y_val!=0].cpu().detach().numpy(),preds[y_val!=0].cpu().detach().numpy() ) ) )
      print('\n')

  model.eval()
  preds = model(X_val,A_val,We_val,S_val,H_val)
  rmse_dict[store]=(((preds-y_val)**2)[y_val!=0].mean()**0.5).item()
  if se is None:
    se = ((preds-y_val)**2)[y_val!=0]
  else:
    se = torch.cat([se,((preds-y_val)**2)[y_val!=0]])
    




Store #1
Epoch: 0, RMSE test: 10.61642074584961, R2 test: -0.34014841316683175


Epoch: 1, RMSE train: 8.568536758422852, R2 train: 0.333652645106436
Epoch: 1, RMSE test: 7.260906219482422, R2 test: 0.3731288019341401


Epoch: 26, RMSE train: 6.623193264007568, R2 train: 0.6018723933492713
Epoch: 26, RMSE test: 5.538658142089844, R2 test: 0.6352412086724784


Epoch: 50, RMSE train: 6.315693378448486, R2 train: 0.6379825884267798
Epoch: 50, RMSE test: 5.5644001960754395, R2 test: 0.6318427475745136


Store #2
Epoch: 0, RMSE test: 14.637755393981934, R2 test: -0.2717485078474098


Epoch: 1, RMSE train: 11.066709518432617, R2 train: 0.3080476336557203
Epoch: 1, RMSE test: 10.840424537658691, R2 test: 0.3024990534310372


Epoch: 26, RMSE train: 8.062589645385742, R2 train: 0.6327276202893678
Epoch: 26, RMSE test: 8.681868553161621, R2 test: 0.552617856209656


Epoch: 50, RMSE train: 7.779990196228027, R2 train: 0.6580227214605232
Epoch: 50, RMSE test: 8.291024208068848, R2 test: 0.59199207

In [ ]:
print("RMSE val: {}".format( (se.mean()**0.5).item() ))

RMSE val: 18.754392623901367


In [ ]:
print("RMSE на валидационной выборке для различных магазинов:")
for key in rmse_dict:
  print("Store #{}: {}".format(key,round(rmse_dict[key],3)))

RMSE на валидационной выборке для различных магазинов:
Store #1: 5.564
Store #2: 8.291
Store #3: 16.036
Store #4: 6.405
Store #5: 5.943
Store #6: 9.68
Store #7: 8.04
Store #8: 8.535
Store #9: 9.774
Store #10: 14.23
Store #11: 12.11
Store #12: 17.53
Store #13: 9.413
Store #14: 32.593
Store #15: 5.117
Store #16: 15.702
Store #17: 12.572
Store #18: 6.055
Store #19: 8.233
Store #20: 26.408
Store #21: 24.4
Store #22: 11.906
Store #23: 7.249
Store #24: 8.593
Store #25: 5.992
Store #26: 7.222
Store #27: 9.78
Store #28: 10.343
Store #29: 8.616
Store #30: 27.892
Store #31: 34.499
Store #32: 17.637
Store #33: 27.15
Store #34: 8.462
Store #35: 6.4
Store #36: 9.872
Store #37: 10.516
Store #38: 25.927
Store #39: 43.828
Store #40: 41.941
Store #41: 29.109
Store #42: 15.662
Store #43: 14.451
Store #44: 21.282
Store #45: 21.492
Store #46: 18.415
Store #47: 26.67
Store #48: 17.231
Store #49: 12.426
Store #50: 12.176
Store #51: 13.813
Store #52: 10.598
Store #53: 6.711
Store #54: 55.836


Мы видим, что RMSE снизилась по сравнению с бэйзлайном, однако точность прогнозирования сильно зависит от магазина.

Используем данную модель для совершения финальных прогнозов, добавив при этом валидационные данные.

In [16]:
date = (np.array('2017-08-01', dtype=np.datetime64) + np.arange(15)).astype(str)
df_test_transformed = None
for store in df_submission.STORE_ID.unique():
  if df_test_transformed is None:
    df_test_transformed = list( product(date,[store],articles[store] ))
  else:
    df_test_transformed.extend( list( product(date,[store],articles[store] )) )

In [17]:
df_test_transformed = pd.DataFrame(df_test_transformed,columns = ['DAY_ID','STORE_ID','ARTICLE_ID'])
df_test_transformed = pd.merge(df_test_transformed,df_test,how='outer')
df_test_transformed = pd.merge(df_test_transformed,df_sales_count)

In [18]:
df_test_transformed['IS_HOLIDAY'] = df_test_transformed['DAY_ID'].apply(lambda x: x in df_holidays['DAY_ID'].values)
df_test_transformed['WEEKDAY'] = df_test_transformed['DAY_ID'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d').weekday())

In [19]:
df_test_transformed.fillna(0,inplace=True)
df_test_transformed.sort_values(by = ['DAY_ID','STORE_ID','ARTICLE_ID'],inplace=True)

In [20]:
df_test_transformed['SALES'] = 0.

In [21]:

for store in res.STORE_ID.unique():
  ds = res[res.STORE_ID==store]
  ds.sort_values(by=['DAY_ID','ARTICLE_ID'],inplace=True)

  ids = ds.ARTICLE_ID.unique()
  ids.sort()
  N_items = ids.shape[0]
  id_int = {ids[i]:i for i in range(N_items)}
  int_id = {i:ids[i] for i in range(N_items)}

  ds['ind'] = ds['ARTICLE_ID'].apply(lambda x: id_int[x])
  ds['WEEKDAY'] = ds['WEEKDAY']*N_items+ds['ind']
  ds['IS_ACTION'] = ds['IS_ACTION']*ds['ind']
  ds['IS_HOLIDAY'] = ds['IS_HOLIDAY']*ds['ind']


  TS = ds.SALES.values.reshape(-1,N_items)

  A = ds.IS_ACTION.values.reshape(-1,N_items)
  We = ds.WEEKDAY.values.reshape(-1,N_items)
  Sa = ds.SALES_COUNT.values.reshape(-1,N_items)
  H = ds.IS_HOLIDAY.values.reshape(-1,N_items)
  
  X,y,A,We,S,H = inout_seq(TS,A,We,Sa,H,train_len,pred_len)

  X,y,A,We,S,H = from_numpy(X).to(device).float(),from_numpy(y).to(device).float(),from_numpy(A).to(device).long(),from_numpy(We).to(device).long(), \
  from_numpy(S).to(device).float(),from_numpy(H).to(device).long()

  model = LSTM(input_size=N_items+1, hidden_size=hidden_size, output_size=N_items, pred_len=pred_len,num_layers=num_layers,dropout=dropout).to(device)
  opt = torch.optim.Adam(model.parameters(),lr=lr)
  N = X.shape[0]
  crit = nn.MSELoss()
  print("Store #{}".format(store))

  for epoch in range(n_epoches):
    model.train()
    inds = torch.randperm(N).to(device)
    for i in range(N//bs):
      opt.zero_grad()
      X_batch,y_batch,A_batch,We_batch,S_batch,H_batch = X[inds[i*bs:(i+1)*bs]],y[inds[i*bs:(i+1)*bs]],A[inds[i*bs:(i+1)*bs]],We[inds[i*bs:(i+1)*bs]], \
      S[inds[i*bs:(i+1)*bs]], H[inds[i*bs:(i+1)*bs]]

      preds = model(X_batch,A_batch,We_batch,S_batch,H_batch)
      loss = crit(preds*(y_batch!=0),y_batch)
      loss.backward()
      opt.step()

  model.eval()

  ds_test = df_test_transformed[df_test_transformed.STORE_ID==store]
  ds_test.sort_values(by=['DAY_ID','ARTICLE_ID'],inplace=True)

  X_test = torch.unsqueeze(from_numpy(np.concatenate([ TS[-train_len:],Sa[-train_len:,[0]] ],axis=1)).to(device).float(),0)
  A_test = torch.unsqueeze(from_numpy(ds_test.IS_ACTION.values.reshape(-1,N_items)).to(device).long(),0)
  We_test = torch.unsqueeze(from_numpy(ds_test.WEEKDAY.values.reshape(-1,N_items)).to(device).long(),0)
  S_test = torch.unsqueeze(from_numpy(ds_test.SALES_COUNT.values.reshape(-1,N_items)).to(device).float(),0)
  H_test = torch.unsqueeze(from_numpy(ds_test.IS_ACTION.values.reshape(-1,N_items)).to(device).long(),0)

  preds = model(X_test,A_test,We_test,S_test,H_test)[0].cpu().detach().numpy().reshape(-1)
  df_test_transformed.loc[df_test_transformed.STORE_ID==store,'SALES'] = preds
  

Store #1
Store #2
Store #3
Store #4
Store #5
Store #6
Store #7
Store #8
Store #9
Store #10
Store #11
Store #12
Store #13
Store #14
Store #15
Store #16
Store #17
Store #18
Store #19
Store #20
Store #21
Store #22
Store #23
Store #24
Store #25
Store #26
Store #27
Store #28
Store #29
Store #30
Store #31
Store #32
Store #33
Store #34
Store #35
Store #36
Store #37
Store #38
Store #39
Store #40
Store #41
Store #42
Store #43
Store #44
Store #45
Store #46
Store #47
Store #48
Store #49
Store #50
Store #51
Store #52
Store #53
Store #54


In [22]:
final = pd.merge(df_test_transformed[['DAY_ID','STORE_ID','ARTICLE_ID','SALES']],df_test[['DAY_ID','STORE_ID','ARTICLE_ID']],how='inner')

In [23]:
final.to_csv('/content/drive/MyDrive/regression/final_reg.csv')